In [ ]:
import cv2
import numpy as np

# initialize webcam
cap = cv2.VideoCapture(1)

# define skin color range in HSV
lower_skin = np.array([0, 20, 70], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # flip the frame for a mirror effect
    frame = cv2.flip(frame, 1)

    # define Region of Interest 
    roi = frame[100:400, 100:400]

    # converted ROI to HSV and apply skin color mask
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # appled Gaussian blur and morphological operations for noise reduction
    skin_mask = cv2.GaussianBlur(skin_mask, (5, 5), 0)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
    skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)

    # find contours from the skin mask
    contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # find the largest contour and apply area threshold
        max_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(max_contour) > 2000: 
            # draw the contour and convex hull on the ROI
            cv2.drawContours(roi, [max_contour], -1, (0, 255, 0), 2)
            hull = cv2.convexHull(max_contour, returnPoints=False)

            if hull is not None and len(hull) > 3:
                defects = cv2.convexityDefects(max_contour, hull)
                if defects is not None:
                    for i in range(defects.shape[0]):
                        start_idx, end_idx, far_idx, _ = defects[i, 0]
                        start = tuple(max_contour[start_idx][0])
                        end = tuple(max_contour[end_idx][0])
                        far = tuple(max_contour[far_idx][0])

                        # draw the start, end, and far points for defects
                        cv2.circle(roi, start, 5, (0, 255, 0), -1)
                        cv2.circle(roi, end, 5, (0, 0, 255), -1)
                        cv2.circle(roi, far, 5, (255, 0, 0), -1)
                        cv2.line(roi, start, end, (0, 255, 255), 2)


    cv2.imshow("Skin Mask", skin_mask)
    cv2.imshow("Hand Contour", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
